<a href="https://colab.research.google.com/github/newturok/softconstrust_testr/blob/init-branch/data_overview_and_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and setups

Change **project_id** to yours google project (https://console.cloud.google.com/home/dashboard)

In [0]:
project_id = 'spartan-grail-270510'

In [0]:
from pathlib import Path

import numpy as np
import pandas as pd

pd.options.display.float_format = '{:,.3f}'.format

# Data Overview

In [0]:
data_overview_df = pd.read_gbq('''
  SELECT *
  FROM `bigquery-public-data.iowa_liquor_sales.sales` 
  limit 1
''', project_id=project_id)
data_overview_df

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,S22616100020,2014-11-25,4132,Camanche Food Pride,908 7TH AVE,CAMANCHE,52730,POINT (-90.257535 41.788751),23,Clinton,1701100,DECANTERS & SPECIALTY PACKAGES,55,Sazerac North America,2618,99 Family 4-50mls,15,200,2.080,3.120,2,6.240,0.400,0.110


In [0]:
# Prepare strign for SQL request to get unique values counts for each column in df
unique_values_request_part = ', '.join([f'count (distinct {column}) as {column}' for column in data_overview_df.columns])
# Execute that query
unique_values_counts_input = pd.read_gbq(f'SELECT {unique_values_request_part}, count(*) as rows_count FROM `bigquery-public-data.iowa_liquor_sales.sales`')
# Show results
unique_values_counts = unique_values_counts_input.iloc[0].rename('unique_values').reset_index().sort_values('unique_values')

In [0]:
unique_values_counts

,index,unique_values
16,pack,27
17,bottle_volume_ml,55
10,category,106
8,county_number,108
11,category_name,129
9,county,201
12,vendor_number,341
13,vendor_name,461
6,zip_code,486
20,bottles_sold,560


# Data Preparation

## Counties
https://en.wikipedia.org/wiki/List_of_counties_in_Iowa

In [0]:
iowa_counties = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_counties_in_Iowa'
)[1][['#', 'County']].rename(columns = {
    '#': 'county_number_adjusted',
    'County': 'county_name',
})
iowa_counties

,county_number_adjusted,county_name
0,1,Adair County
1,2,Adams County
2,3,Allamakee County
3,4,Appanoose County
4,5,Audubon County
...,...,...
94,95,Winnebago County
95,96,Winneshiek County
96,97,Woodbury County
97,98,Worth County


In [0]:
unique_values_counts[unique_values_counts['index'].str.contains('county')]

,index,unique_values
8,county_number,108
9,county,201


In [0]:
counties_input = pd.read_gbq(f'''
  SELECT 
  county_number, county, sum(sale_dollars) as sale_dollars
  FROM `bigquery-public-data.iowa_liquor_sales.sales` 
  group by 1, 2
''', project_id=project_id)

In [0]:
counties_input

,county_number,county,sale_dollars
0,75,Plymouth,"9,596,915.020"
1,10,Buchanan,"6,532,985.710"
2,52,Johnson,"75,137,953.740"
3,77,Polk,"299,839,327.120"
4,85,Story,"38,330,779.550"
...,...,...,...
200,87,TAYLOR,"430,389.770"
201,78,Pottawatta,"181,114.530"
202,1,ADAIR,"859,809.810"
203,36,FREMONT,"177,711.720"


In [0]:
counties_eda = counties_input.sort_values('sale_dollars', ascending=False).assign(
    market_share = lambda x: x.sale_dollars / x.sale_dollars.sum(),
    cummulative_market_share = lambda x: x.market_share.cumsum(),
    names_count = lambda x: x.groupby('county_number').county.transform('count'),
)
counties_eda.query('names_count > 2')

,county_number,county,sale_dollars,market_share,cummulative_market_share,names_count
80,78,Pottawattamie,"44,123,018.410",0.018,0.518,3.000
14,78,POTTAWATTA,"40,032,896.940",0.017,0.552,3.000
119,17,Cerro Gordo,"25,766,350.550",0.011,0.655,3.000
13,17,CERRO GORD,"18,266,044.540",0.008,0.678,3.000
125,17,Cerro Gord,"5,171,329.210",0.002,0.878,3.000
201,78,Pottawatta,"181,114.530",0.000,1.000,3.000


In [0]:
counties_eda = counties_input.dropna().assign(
    county_number_adjusted = lambda x: x.county_number.astype(int)
).groupby(['county_number', 'county_number_adjusted']).sale_dollars.sum().reset_index().merge(
    iowa_counties, on='county_number_adjusted', how='left'
).assign(
    county_name = lambda x: x.county_name.str.replace(' County', '')
)
counties_eda

,county_number,county_number_adjusted,sale_dollars,county_name
0,01,1,"1,909,292.350",Adair
1,02,2,"434,913.150",Adams
2,03,3,"3,764,465.400",Allamakee
3,04,4,"3,609,724.530",Appanoose
4,05,5,"763,133.410",Audubon
...,...,...,...,...
103,95,95,"5,933,873.940",Winnebago
104,96,96,"10,960,438.340",Winneshiek
105,97,97,"82,038,239.880",Woodbury
106,98,98,"2,008,562.140",Worth


In [0]:
counties_mapping = counties_eda[['county_number', 'county_name']]

In [0]:
counties_mapping.to_gbq('mappings.counties_mapping', project_id=project_id, if_exists='replace')

1it [00:03,  3.01s/it]


## Bottle Volume

In [0]:
bottle_volume_input = pd.read_gbq(f'''
  SELECT 
  bottle_volume_ml, sum(sale_dollars) as sale_dollars
  FROM `bigquery-public-data.iowa_liquor_sales.sales` 
  group by 1
''', project_id=project_id)

In [0]:
bottle_volume_eda = bottle_volume_input.sort_values('sale_dollars', ascending=False).assign(
    market_share = lambda x: x.sale_dollars / x.sale_dollars.sum(),
    cummulative_market_share = lambda x: x.market_share.cumsum(),
)
bottle_volume_eda.head(10)

,bottle_volume_ml,sale_dollars,market_share,cummulative_market_share
1,750,"963,061,056.780",0.403,0.403
6,1750,"606,003,060.579",0.254,0.656
5,1000,"569,752,886.130",0.238,0.895
3,375,"131,034,657.530",0.055,0.950
15,200,"50,404,414.860",0.021,0.971
13,500,"22,045,334.320",0.009,0.980
29,3000,"11,644,790.480",0.005,0.985
20,100,"9,192,985.860",0.004,0.989
9,600,"8,598,115.370",0.004,0.992
4,50,"6,340,007.160",0.003,0.995


In [0]:
bottle_volume_mapping = bottle_volume_eda.assign(
    bottle_volume_type = lambda x: np.where(
        x.cummulative_market_share < 0.95,
        'Mainstream',
        'Special',
    ),
    bottle_volume_size = lambda x: np.where(
        x.bottle_volume_ml < 1000,
        '< 1L',
        '>= 1L',
    ),
)
bottle_volume_mapping.head(10)

,bottle_volume_ml,sale_dollars,market_share,cummulative_market_share,bottle_volume_type,bottle_volume_size
1,750,"963,061,056.780",0.403,0.403,Mainstream,< 1L
6,1750,"606,003,060.579",0.254,0.656,Mainstream,>= 1L
5,1000,"569,752,886.130",0.238,0.895,Mainstream,>= 1L
3,375,"131,034,657.530",0.055,0.950,Mainstream,< 1L
15,200,"50,404,414.860",0.021,0.971,Special,< 1L
13,500,"22,045,334.320",0.009,0.980,Special,< 1L
29,3000,"11,644,790.480",0.005,0.985,Special,>= 1L
20,100,"9,192,985.860",0.004,0.989,Special,< 1L
9,600,"8,598,115.370",0.004,0.992,Special,< 1L
4,50,"6,340,007.160",0.003,0.995,Special,< 1L


In [0]:
# Save mapping to GoogleBigQuery Table
bottle_volume_mapping[[
    'bottle_volume_ml', 'bottle_volume_type', 'bottle_volume_size'
]].to_gbq('mappings.bottle_volume_mapping', project_id=project_id, if_exists='replace')

1it [00:02,  2.97s/it]


## Categories

In [0]:
unique_values_counts[unique_values_counts['index'].str.contains('category')]

,index,unique_values
10,category,106
11,category_name,129


In [0]:
category_input = pd.read_gbq(f'''
  SELECT 
  category, category_name, sum(sale_dollars) as sale_dollars
  FROM `bigquery-public-data.iowa_liquor_sales.sales` 
  group by 1, 2
''', project_id=project_id)

In [0]:
category_eda = category_input.sort_values('sale_dollars', ascending=False).assign(
    market_share = lambda x: x.sale_dollars / x.sale_dollars.sum(),
    cummulative_market_share = lambda x: x.market_share.cumsum(),
    category_names_count = lambda x: x.groupby('category').category_name.transform('count'),
    category_code = lambda x: x.category.str[:3],
)

In [0]:
category_eda.query('category_names_count > 1').sort_values('category').tail(15)

,category,category_name,sale_dollars,market_share,cummulative_market_share,category_names_count,category_code
99,1062300,FLAVORED RUM,"27,277,184.180",0.011,0.742,2.000,106
103,1071100,Cocktails /RTD,"17,615,034.590",0.007,0.883,2.000,107
129,1071100,AMERICAN COCKTAILS,"22,720,367.500",0.010,0.812,2.000,107
102,1081200,CREAM LIQUEURS,"34,183,197.930",0.014,0.669,2.000,108
104,1081200,Cream Liqueurs,"23,277,979.970",0.010,0.793,2.000,108
132,1081300,PEPPERMINT SCHNAPPS,"6,163,636.860",0.003,0.967,2.000,108
140,1081300,American Cordials & Liqueur,"11,778,152.770",0.005,0.942,2.000,108
18,1081400,TRIPLE SEC,"3,246,462.930",0.001,0.980,2.000,108
121,1081400,American Schnapps,"18,451,175.910",0.008,0.846,2.000,108
0,1081500,Triple Sec,"2,542,629.460",0.001,0.984,2.000,108


As we see names for category code are non unique.  
Along with that categories can be aggrigated to more general categories by first 3 charackters of code

To detect general categories names we will find most popular word foreach category_code

In [0]:
 general_category_potential_names = (
    category_eda.set_index('category_code')
    .category_name.str.split(' ', expand=True).unstack().rename('word').str.upper().reset_index(level=1)
    .groupby(['category_code', 'word']).size().rename('word_count').reset_index()
    .assign(
        word_count_rank = lambda x: x.groupby('category_code')['word_count'].rank("first", ascending=False)
    )
    .query('word_count_rank == 1')
).merge(
    category_eda.groupby('category_code').sale_dollars.sum().reset_index(),
    on='category_code'
).sort_values('sale_dollars', ascending=False)
 general_category_potential_names

,category_code,word,word_count,word_count_rank,sale_dollars
0,101,WHISKIES,19,1.000,"743,835,980.800"
2,103,VODKA,12,1.000,"574,356,854.210"
7,108,SCHNAPPS,18,1.000,"381,108,968.150"
5,106,RUM,10,1.000,"296,831,409.560"
1,102,TEQUILA,3,1.000,"136,365,572.230"
4,105,BRANDIES,9,1.000,"103,764,873.310"
3,104,GINS,7,1.000,"62,764,271.660"
6,107,COCKTAILS,3,1.000,"40,385,427.740"
11,170,&,3,1.000,"29,767,711.690"
8,109,DISTILLED,4,1.000,"9,187,456.440"


Everything that has marketshare > 30 mln has correct names, other must be adjusted by hand

So we will see all that items and adjust it

In [0]:
category_eda[category_eda['category_code'].isin(
    general_category_potential_names.query('sale_dollars < 30000000').category_code
)].sort_values('category_code', ascending=False)

,category,category_name,sale_dollars,market_share,cummulative_market_share,category_names_count,category_code
31,1901200,Special Order Items,"5,839,905.630",0.002,0.969,1.000,190
90,1901100,Delisted Items,"4,589.920",0.000,1.000,1.000,190
1,1701100,Temporary & Specialty Packages,"17,559,180.990",0.007,0.891,2.000,170
4,1701100,DECANTERS & SPECIALTY PACKAGES,"12,166,703.710",0.005,0.938,2.000,170
71,1701200,Holiday VAP,"3,637.520",0.000,1.000,1.000,170
69,1700000,Temporary & Specialty Packages,"38,189.470",0.000,1.000,1.000,170
92,1501100,HIGH PROOF BEER - AMERICAN,"6,237.840",0.000,1.000,1.000,150
3,1101100,AMERICAN ALCOHOL,"2,966,946.760",0.001,0.981,1.000,110
38,1091100,American Distilled Spirit Specialty,"990,639.040",0.000,0.994,1.000,109
48,1091300,Neutral Grain Spirits Flavored,"978,053.020",0.000,0.994,1.000,109


In [0]:
category_by_code_mapping = general_category_potential_names.assign(
    category_name = lambda x: np.where(
        x.sale_dollars > 30_000_000,
        x.word.str.capitalize().replace({'Schnapps': 'Liqueurs'}),
        x.category_code.replace({
            '170': 'Specialty Packages',
            '109': 'Others',
            '190': 'Others',
            '110': 'Others',
            '150': 'Others',
        })
    ),
    category_size = lambda x: np.where(
        x.sale_dollars > 100_000_000,
        'Top Tier',
        'Second Tier',
    ),
)
category_by_code_mapping

,category_code,word,word_count,word_count_rank,sale_dollars,category_name,category_size
0,101,WHISKIES,19,1.000,"743,835,980.800",Whiskies,Top Tier
2,103,VODKA,12,1.000,"574,356,854.210",Vodka,Top Tier
7,108,SCHNAPPS,18,1.000,"381,108,968.150",Liqueurs,Top Tier
5,106,RUM,10,1.000,"296,831,409.560",Rum,Top Tier
1,102,TEQUILA,3,1.000,"136,365,572.230",Tequila,Top Tier
4,105,BRANDIES,9,1.000,"103,764,873.310",Brandies,Top Tier
3,104,GINS,7,1.000,"62,764,271.660",Gins,Second Tier
6,107,COCKTAILS,3,1.000,"40,385,427.740",Cocktails,Second Tier
11,170,&,3,1.000,"29,767,711.690",Specialty Packages,Second Tier
8,109,DISTILLED,4,1.000,"9,187,456.440",Others,Second Tier


In [0]:
category_mapping = category_eda.groupby(['category', 'category_code']).size().reset_index().merge(
    category_by_code_mapping[['category_code', 'category_name', 'category_size']], on='category_code'
)[['category', 'category_name', 'category_size']]
category_mapping

,category,category_name,category_size
0,1011100,Whiskies,Top Tier
1,1011200,Whiskies,Top Tier
2,1011250,Whiskies,Top Tier
3,1011300,Whiskies,Top Tier
4,1011400,Whiskies,Top Tier
...,...,...,...
101,1700000,Specialty Packages,Second Tier
102,1701100,Specialty Packages,Second Tier
103,1701200,Specialty Packages,Second Tier
104,1901100,Others,Second Tier


In [0]:
# Save mapping to GoogleBigQuery Table
category_mapping.to_gbq('mappings.category_mapping', project_id=project_id, if_exists='replace')

1it [00:03,  3.07s/it]


## Vendors

In [0]:
unique_values_counts[unique_values_counts['index'].str.contains('vendor')]

,index,unique_values
12,vendor_number,341
13,vendor_name,461


In [0]:
vendor_input = pd.read_gbq(f'''
  SELECT 
  vendor_number, vendor_name, sum(sale_dollars) as sale_dollars, max(date) as last_date
  FROM `bigquery-public-data.iowa_liquor_sales.sales` 
  group by 1, 2
''', project_id=project_id)

In [0]:
vendor_eda = vendor_input.sort_values('sale_dollars', ascending=False).assign(
    vendor_number_adjusted = lambda x: np.where(
        x.vendor_number.str.len() == 2,
        '0' + x.vendor_number,
        x.vendor_number
    ),
    market_share = lambda x: x.sale_dollars / x.sale_dollars.sum(),
    cummulative_market_share = lambda x: x.market_share.cumsum(),
    names_count = lambda x: x.groupby('vendor_number_adjusted').vendor_name.transform('count'),
    name_rank = lambda x: x.groupby('vendor_number_adjusted').last_date.rank(ascending=False)
)

Some vendor numbers has more than 1 vendor name.
Typical reasons are:
- different writing styles, like: **LEVECKE CORP** -> **Levecke Corp.** -> **Levecke Corporation**
- real name changes, like **Sidney Frank Importing Company, Inc.** -> **Mast-Jagermeister US, Inc**: https://www.prnewswire.com/news-releases/sidney-frank-importing-company-announces-name-change-to-mast-jagermeister-us-300426182.html  

So we clean it to unique value for number and use last inserted name as actual  
Also there are some vendors that has vendor_number with and without leading 0

In [0]:
vendor_eda.query('names_count > 3 | vendor_number_adjusted in ["065", "085", "035", "055"]').sort_values('vendor_number_adjusted')

,vendor_number,vendor_name,sale_dollars,last_date,vendor_number_adjusted,market_share,cummulative_market_share,names_count,name_rank
41,35,"Bacardi U.S.A., Inc.","67,172,214.940",2016-08-26,035,0.028,0.597,2.000,2.000
8,035,BACARDI USA INC,"45,961,384.670",2020-01-31,035,0.019,0.732,2.000,1.000
10,55,Sazerac North America,"46,026,495.950",2016-08-26,055,0.019,0.713,2.000,2.000
17,055,SAZERAC NORTH AMERICA,"37,265,601.830",2020-01-31,055,0.016,0.781,2.000,1.000
3,65,Jim Beam Brands,"100,221,573.550",2016-08-26,065,0.042,0.259,2.000,2.000
7,065,Jim Beam Brands,"86,039,444.080",2020-01-31,065,0.036,0.411,2.000,1.000
26,85,Brown-Forman Corporation,"80,920,329.500",2016-08-26,085,0.034,0.479,2.000,2.000
47,085,Brown Forman Corp.,"59,668,477.380",2020-01-31,085,0.025,0.674,2.000,1.000
210,192,"Mast-Jaegermeister US, Inc","10,982.190",2018-03-22,192,0.000,1.000,4.000,2.000
341,192,"Sidney Frank Importing Company, Inc.","36,209.220",2017-03-27,192,0.000,0.999,4.000,3.000


In [0]:
vendor_mapping = vendor_eda.groupby(['vendor_number', 'vendor_number_adjusted']).size().reset_index().merge(
    vendor_eda.query('name_rank == 1')[['vendor_number_adjusted', 'vendor_name']], on='vendor_number_adjusted'
)
vendor_mapping

,vendor_number,vendor_number_adjusted,0,vendor_name
0,010,010,1,A HARDY USA LTD
1,10,010,1,A HARDY USA LTD
2,033,033,1,BMC Imports
3,33,033,1,BMC Imports
4,035,035,1,BACARDI USA INC
...,...,...,...,...
336,969,969,2,A V BRANDS INC
337,971,971,2,Hood River Distillers
338,977,977,2,WERNER DISTILLING
339,978,978,2,W J Deutsch & Sons LTD


In [0]:
# Check that there is unique name for unique adjusted number
vendor_mapping.vendor_number_adjusted.nunique(), vendor_mapping.vendor_name.nunique()

(331, 331)

In [0]:
# Save mapping to GoogleBigQuery Table
vendor_mapping[['vendor_number', 'vendor_name']].to_gbq('mappings.vendor_mapping', project_id=project_id, if_exists='replace')

1it [00:03,  3.09s/it]


## Categories Concentration

In [0]:
vendors_by_category_input = pd.read_gbq(f'''
  SELECT 
  vendor_number, category, EXTRACT(YEAR from date) as year, sum(sale_dollars) as sale_dollars
  FROM `bigquery-public-data.iowa_liquor_sales.sales` 
  group by 1, 2, 3
''', project_id=project_id)

In [0]:
def hh_concentration(index_value):
    if index_value < 0.1:
      return 'highly competitive industry'
    elif index_value < 0.15:
      return 'unconcentrated industry'
    elif index_value < 0.25:
     return 'moderate concentration'
    else:
      return 'high concentration' 

def get_hh_index(input_df, year):
    df = (
        input_df.query('year == @year')
        .merge(vendor_mapping[['vendor_number', 'vendor_name']], on='vendor_number')
        .merge(category_mapping, on='category')
        .groupby(['category_name', 'vendor_name'])['sale_dollars'].sum().round().reset_index()
        .assign(vendor_in_group_rank = lambda x: x.groupby('category_name')['sale_dollars'].rank(ascending=False))
        .sort_values(['category_name', 'vendor_in_group_rank'])
        .assign(
            vendor_in_group_share = lambda x: x['sale_dollars'] / x.groupby('category_name')['sale_dollars'].transform('sum'),
            vendor_in_group_cum_share = lambda x: x.groupby('category_name')['sale_dollars'].cumsum() / x.groupby('category_name')['sale_dollars'].transform('sum'),
            vendor_in_group_share_square = lambda x: x.vendor_in_group_share * x.vendor_in_group_share,
        )
    )
    hh_index = df.groupby(['category_name']).vendor_in_group_share_square.sum().round(3).rename(f'hh_index_{year}').sort_values(ascending=False).reset_index()
    hh_index[f'hh_description_{year}'] = hh_index[f'hh_index_{year}'].apply(hh_concentration)

    return hh_index

### [Herfindahl–Hirschman Index](https://en.wikipedia.org/wiki/Herfindahl%E2%80%93Hirschman_Index)
The Herfindahl Index (H) ranges from 1/N to one, where N is the number of firms in the market.

An H below 0.01 indicates a highly competitive industry.  
An H below 0.15 indicates an unconcentrated industry.  
An H between 0.15 to 0.25 indicates moderate concentration.  
An H above 0.25 indicates high concentration.  

In [0]:
year1 = 2016
year2 = 2019

hh_index_mapping = pd.merge(
    get_hh_index(vendors_by_category_input, year1),
    get_hh_index(vendors_by_category_input, year2),
    on='category_name'
).assign(
    trend = lambda x: np.where(
        (x[f'hh_description_{year1}'] != x[f'hh_description_{year2}']) & (x[f'hh_index_{year1}'] > x[f'hh_index_{year2}']),
        'concentration decreased',
        np.where(
            (x[f'hh_description_{year1}'] != x[f'hh_description_{year2}']) & (x[f'hh_index_{year1}'] < x[f'hh_index_{year2}']),
            'concentration increased',
            'no changes'
        )
    )
).sort_values(f'hh_index_{year2}')
hh_index_mapping

,category_name,hh_index_2016,hh_description_2016,hh_index_2019,hh_description_2019,trend
9,Vodka,0.093,highly competitive industry,0.100,unconcentrated industry,concentration increased
7,Whiskies,0.149,unconcentrated industry,0.137,unconcentrated industry,no changes
6,Tequila,0.178,moderate concentration,0.159,moderate concentration,no changes
5,Gins,0.179,moderate concentration,0.169,moderate concentration,no changes
3,Others,0.220,moderate concentration,0.217,moderate concentration,no changes
4,Liqueurs,0.190,moderate concentration,0.236,moderate concentration,no changes
1,Rum,0.298,high concentration,0.271,high concentration,no changes
2,Brandies,0.229,moderate concentration,0.280,high concentration,concentration increased
8,Specialty Packages,0.128,unconcentrated industry,0.343,high concentration,concentration increased
0,Cocktails,0.393,high concentration,0.377,high concentration,no changes


In [0]:
hh_index_mapping.to_gbq('mappings.hh_index_mapping', project_id=project_id, if_exists='replace')

1it [00:02,  2.30s/it]


## Stores

Search for retail networks

In [0]:
unique_values_counts[unique_values_counts['index'].str.contains('store')]

,index,unique_values
7,store_location,1867
2,store_number,2320
3,store_name,2470


In [0]:
store_input = pd.read_gbq(f'''
  SELECT 
  store_number, store_name, sum(sale_dollars) as sale_dollars
  FROM `bigquery-public-data.iowa_liquor_sales.sales` 
  group by 1, 2
''', project_id=project_id)

In [0]:
def has_first_word_in_previous_or_next(df_row):
    row = df_row.str.lower()
    first_word = row.first_word_in_store_name
    second_word = row.second_word_in_store_name
    return (first_word in row.previous_store_name or first_word in row.next_store_name) and (second_word in row.previous_store_name or second_word in row.next_store_name)

In [0]:
store_eda = store_input.assign(
    store_name = lambda x: x.store_name.str.replace('"|,|-|/', ' ').str.replace("'", ' ').str.replace("#", '# ')
).sort_values('store_name', ascending=False).assign(
    market_share = lambda x: x.sale_dollars / x.sale_dollars.sum(),
    names_count = lambda x: x.groupby('store_number').store_name.transform('count'),
    previous_store_name = lambda x: x.store_name.shift(-1).fillna(''),
    next_store_name = lambda x: x.store_name.shift(1).fillna(''),
    first_word_in_store_name = lambda x: x.store_name.str.split(' ').str[0].fillna(''),
    second_word_in_store_name = lambda x: x.store_name.str.split(' ').str[1].fillna(''),
    has_first_word_in_previous_or_next = lambda x: x.apply(has_first_word_in_previous_or_next, axis=1)
)
store_eda.sort_values('store_name').head(5)

,store_number,store_name,sale_dollars,market_share,names_count,previous_store_name,next_store_name,first_word_in_store_name,second_word_in_store_name,has_first_word_in_previous_or_next
1920,5876,Da Booze Barn West Bend,"41,715.630",0.000,1,,Double D Liquor Store,,Da,False
1470,2178,Double D Liquor Store,"416,938.240",0.000,3,Da Booze Barn West Bend,10th Hole Inn & Suite Gift Shop,,Double,False
2114,5675,10th Hole Inn & Suite Gift Shop,"7,109.420",0.000,1,Double D Liquor Store,1st Stop Beverage Shop,10th,Hole,False
215,5443,1st Stop Beverage Shop,"1,068,831.770",0.000,1,10th Hole Inn & Suite Gift Shop,218 Fuel Express,1st,Stop,False
1916,5736,218 Fuel Express,"182,858.780",0.000,1,1st Stop Beverage Shop,218 Fuel Express & Chubby s Liquor,218,Fuel,True


In [0]:
network_mapping_dict = {
    "Casey's": "Casey's Network",
    'Hy-Vee': 'Hy-Vee Network',
    'Kum': 'Kum & Go Network',
    'Fareway': 'Fareway Network',
    'Walgreens': 'Walgreens Network',
    'Wal-Mart': 'Wal-Mart Network',
    'Smokin': "Smokin Joe's Tobacco and Liquor Network",
    'New Star': 'New Star Network',
    'Hometown Foods': 'Hometown Foods Network',
    'Target': 'Target Network',
    'Yesway': 'Yesway Store Network',
    'CVS': 'CVS Pharmacy Network',
    'Quik Trip': 'Quik Trip Network',
    'Kwik': 'Kwik Network',
    "Sam's": "Sam's Network",
    'Prime Mart': 'Prime Mart Network',
    "Dahl": "Dahl's Network",
    'Hartig Drug': 'Hartig Drug Network',
    'Frills': 'No Frills Supermarkets Network',
    "Jeff's Market": "Jeff's Market Network",
    'Hawkeye Convenience Stores': 'Hawkeye Convenience Stores Network',
    'Costco Wholesale': 'Costco Wholesale Network',
    'Wilkie Liquors': 'Wilkie Liquors Network',
    'Price Chopper': 'Price Chopper Network',
    'Liquor Downtown': 'Liquor Downtown Network',
    'Uptown Liquor': 'Uptown Liquor Network',
    'Super Saver': 'Super Saver Network',
    'Lake Liquors Wine and Spirits': 'Lake Liquors Wine and Spirits Network',
}

store_mapping = store_input.copy()
store_mapping['is_network'] = False
store_mapping['network_name'] = store_mapping['store_name']
for search_term, network_name in network_mapping_dict.items():
    store_mapping['network_name'] = np.where(
        store_mapping.store_name.str.lower().str.contains(search_term.lower()),
        network_name,
        store_mapping.network_name,
    )
    store_mapping['is_network'] = np.where(
        store_mapping.store_name.str.lower().str.contains(search_term.lower()),
        True,
        store_mapping.is_network,
    )
store_mapping.head(1)

,store_number,store_name,sale_dollars,is_network,network_name
0,4829,Central City 2,"67,332,563.990",False,Central City 2


In [0]:
store_mapping.is_network.sum(), len(store_mapping)

(1457, 2564)

In [0]:
store_mapping_output = store_mapping.groupby(['store_number', 'network_name', 'is_network']).sale_dollars.sum().reset_index().assign(
    market_share_rank = lambda x: x.groupby('store_number')['sale_dollars'].rank("first", ascending=False)
).query('market_share_rank == 1')[['store_number', 'network_name', 'is_network']]
store_mapping_output.query('store_number == "4152"')

,store_number,network_name,is_network
615,4152,Food Land Super Markets / Missouri V,False


In [0]:
# Save mapping to GoogleBigQuery Table
store_mapping_output.to_gbq('mappings.store_mapping', project_id=project_id, if_exists='replace')

1it [00:03,  3.76s/it]
